Librerias

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

Carga del dataset

In [41]:
df = pd.read_csv('data/medellin_properties.csv')

In [42]:
df.head()

,neighbourhood,latitude,longitude,property_type,price,rooms,baths,area,administration_price,age,garages,stratum
0,Suramerica,6.1862025,-75.5994371,Apartamento,435000000,3.0,2.0,83.0,354400.0,1.0,1,4
1,Escobero,6.1627997,-75.57351899999999,Apartamento,680000000,3.0,4.0,124.0,480000.0,2.0,2,4
2,Castropol,6.2161401,-75.56697,Apartamento,900000000,3.0,3.0,111.0,813000.0,1.0,2,6
3,Toledo,6.1627622,-75.6393071,Casa,650000000,3.0,2.0,127.0,0.0,2.0,1,4
4,La pilarica,6.2476376,-75.56581530000001,Apartamento,320000000,3.0,2.0,72.0,250000.0,NaN,2,5


- Para el modelo, el `property_type` Proyecto no está en nuestro scope, hay que quitarlo.
- La variable `neighbourhood` es irrelevante (no sé bajo que criterio, seria bueno verlo después)
- En `property_type` Apartaestudio hay que pasarlo a Apartamento.
- Voy a eliminar ciertos valores, como INVALID y Más de 10
- Encoding a las variables categoricas.

In [43]:
df = df.loc[df['property_type']!='Proyecto']
df = df.drop(columns=['neighbourhood','age'])
df['property_type'] = df['property_type'].replace('Apartaestudio','Apartamento')

# latitude creo que si era necesario eliminar los INVALID, pero de garages el eliminar Más de 10 no creo que haya sido correcto.
df = df.loc[df['latitude']!='INVALID']
df = df.loc[df['garages']!='Más de 10']

# Por ahora uso get_dummies, pero es mejor one hot con pipelines
df = pd.get_dummies(df, columns=['property_type'], drop_first=False)*1

Esto me imputa los datos con el model de RF, también termina cambiandolo de tipo de dato, de object a float.

In [44]:
imp = IterativeImputer(estimator=RandomForestRegressor())
df_imp = imp.fit_transform(df)
df = pd.DataFrame(df_imp, columns=df.columns, index=df.index)

En el video se usaba una detección de anomalias con `IsolationForest`, también lo ignoré.

In [45]:
X = df.drop('price', axis=1).copy()
y = df['price'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (8924, 10)
Tamaño del conjunto de prueba: (992, 10)


Escalo los datos

In [46]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

Realizo el modelo

In [47]:
regr = RandomForestRegressor(n_estimators=10, random_state=42, criterion='squared_error')
regr.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [48]:
y_pred = regr.predict(X_test_scaled)
mse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {np.sqrt(mse):,.2f}')

RMSE: 131,042.91


Tuneando el modelo con GridSearchCV

In [49]:
param_grid = {
    'n_estimators': [5, 10, 30, 40, 50, 100, 200, 300],  
    'max_depth': [None, 5, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 3, 5, 10]
}

regr = RandomForestRegressor(random_state=42, criterion='squared_error')

grid_search = GridSearchCV(estimator=regr, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 20, 30, 40, 50],
                         'min_samples_split': [2, 3, 5, 10],
                         'n_estimators': [5, 10, 30, 40, 50, 100, 200, 300]},
             scoring='neg_mean_squared_error')

In [50]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score (negative mean squared error): {grid_search.best_score_}")

Best parameters: {'max_depth': None, 'min_samples_split': 3, 'n_estimators': 5}
Best score (negative mean squared error): -1.544247863517697e+21


In [51]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'RMSE: {np.sqrt(mse):,.2f}')
print(f'MSE: {mse:,.2f}')

RMSE: 13,552,912,086.18
MSE: 183,681,426,015,703,171,072.00


In [52]:
print(f'MSE: {mse:,.2f}')

MSE: 183,681,426,015,703,171,072.00


Utilizando una técnica similar a grid search, pero es bayes search

In [53]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [54]:
param_space = {
    'n_estimators': Integer(10, 100), 
    'max_depth': Integer(1, 50),   
    'min_samples_split': Real(0.01, 1.0, 'uniform'), 
}

regr = RandomForestRegressor(random_state=42, criterion='squared_error')
optimizer = BayesSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    search_spaces=param_space,
    n_iter=25,  
    cv=5,      
    random_state=42
)
optimizer.fit(X_train_scaled, y_train)

BayesSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_iter=25,
              random_state=42,
              search_spaces={'max_depth': Integer(low=1, high=50, prior='uniform', transform='normalize'),
                             'min_samples_split': Real(low=0.01, high=1.0, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=10, high=100, prior='uniform', transform='normalize')})

In [55]:
print(f"Best parameters: {optimizer.best_params_}")
print(f"Best score (negative mean squared error): {optimizer.best_score_}")

Best parameters: OrderedDict([('max_depth', 48), ('min_samples_split', 0.7036152301751524), ('n_estimators', 88)])
Best score (negative mean squared error): -0.4108821309639071


In [56]:
best_model = optimizer.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'RMSE: {np.sqrt(mse):,.2f}')
print(f'MSE: {mse:,.2f}')

RMSE: 6,707,542,618.78
MSE: 44,991,127,982,705,205,248.00


En el video lo intentaban también con `CatBoostRegressor` que es como otro randomforest

voy para el `5_basic-supervised-learning-automl.ipynb`